In [48]:
import datetime

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from elarb import datasets, util

## Load data

In [59]:
#df = datasets.download_nordpool('2022-10-09', '2022-10-11')
#df = df[df.HourDK >= '2022-10-09 12:00:00'].reset_index()
#df.to_csv('../data/example.csv', index=False)
df = pd.read_csv('../data/example.csv')
spot_dkk = df.SpotPriceDKK.values / 1000.0
spot_dkk

array([0.44604001, 0.37189002, 0.40722   , 0.38260001, 0.98460999,
       1.36169995, 1.47892004, 1.92443994, 1.07883997, 0.70509998,
       0.68940002, 0.45995001, 0.51596002, 0.40185998, 0.07274   ,
       0.06716   , 0.07088   , 0.08412   , 0.95025   , 1.69030005,
       1.98036999, 1.61532996, 1.22744995, 1.18163   , 0.99264001,
       0.97494   , 0.97560999, 1.03659998, 1.18156006, 1.56623999,
       1.84315002, 2.05832007, 1.33432996, 1.17464002, 1.15581995,
       0.89945001])

## Optimal policy

```
           ┌───────┐       
    ┌─x1──▶│ Grid  │──┐    
    │      └───────┘  │    
┌───────┐      ▲      │    
│ Solar │      │x3    │x4    
└───────┘      │      │    
    │      ┌───────┐  │    
    └─x2──▶│Battery│◀─┘    
           └───────┘       
```

### Parameters

In [177]:
n = len(spot_dkk)

# 1 inverter: Deye SUN-12K-SG04LP3-EU
# ... ignored

# grid
standard_outlet_kWh = 12.8
d_grid = cp.Parameter(n, nonneg=True)
d_grid.value = np.zeros(n) + standard_outlet_kWh
s_grid = cp.Parameter(n, nonneg=True)
s_grid.value = np.zeros(n) + standard_outlet_kWh

# 1 battery: Powerwall LBATTS Powerwall, 8.8 kwh
bat_cap = 26.4
bat_discharge = 11.52
bat_depreciation = 0.3
bat_loss = 0.03
bat_SoC = ... # defined later

# 33 solar panels: 470Wp JinKO TigerNeo N-Type 60HL4 BF, 1.57 kWh / time
solar_kWh = 1.57
solar_depreciation = 0.16
s_solar = cp.Parameter(n, nonneg=True)
s_solar.value = np.zeros(n) + solar_kWh

# prices
p_x1 = cp.Parameter(n)
p_x1.value = np.zeros(n) + spot_dkk - solar_depreciation 

p_x2 = cp.Parameter(n)
p_x2.value = np.zeros(n) - solar_depreciation # - bat_depreciation

p_x3 = cp.Parameter(n)
p_x3.value = np.zeros(n) + spot_dkk - bat_depreciation

p_x4 = cp.Parameter(n)
p_x4.value = np.zeros(n) - spot_dkk - bat_depreciation

### Variables

In [178]:
x1 = cp.Variable(n, nonneg=True)
x2 = cp.Variable(n, nonneg=True)
x3 = cp.Variable(n, nonneg=True)
x4 = cp.Variable(n, nonneg=True)

### Objective function

In [179]:
contrib_x1 = p_x1@x1
contrib_x2 = p_x2@x2
contrib_x3 = p_x3@x3
contrib_x4 = p_x4@x4
objective = cp.Maximize(cp.sum(contrib_x1 + contrib_x2 + contrib_x3 + contrib_x4))

### Constraints

```
           ┌───────┐       
    ┌─x1──▶│ Grid  │──┐    
    │      └───────┘  │    
┌───────┐      ▲      │    
│ Solar │      │x3    │x4    
└───────┘      │      │    
    │      ┌───────┐  │    
    └─x2──▶│Battery│◀─┘    
           └───────┘       
```

In [180]:
# TODO: plug in battery loss somewhere... in bat_SoC
charge_loss = 0.97
bat_soc = cp.cumsum(x2*charge_loss + x4*charge_loss - x3)
d_bat = bat_cap - bat_soc

constraints = [
    # sales to grid cannot exceed demand of grid
    x1 + x3 <= d_grid,  
    # sales to bat cannot exceed demand (i.e. gap to full) of battery
    x2 + x4 <= d_bat, 
    # buys from solar cannot exceed supply of solar
    x1 + x2 <= s_solar,
    # sells from battery cannot exceed maximum discharge
    x3 <= bat_discharge,
    # sells from battery cannot exceed state of charge (SoC)  
    x3 <= bat_soc,
    # buys from grid cannot exceed supply of grid
    x4 <= s_grid,
    # SoC cannot exceed battery capacity, i.e. the max over cumulative sum of battery flow cannot exceed capacity 
    cp.max(bat_soc) <= bat_cap
]

### Solve problem

In [181]:
prob = cp.Problem(objective, constraints)
result = prob.solve()
result_solaronly = np.round(cp.sum(s_solar@p_x1).value, 2)
result_solar_and_arb = np.round(result, 2)

x1_component = cp.sum(x1@p_x1).value.round(2)
x2_component = cp.sum(x2@p_x2).value.round(2)
x3_component = cp.sum(x3@p_x3).value.round(2)
x4_component = cp.sum(x4@p_x4).value.round(2)

print('Solar only =', result_solaronly, 'DKK')
print('Solar + arbitrage =', result_solar_and_arb, 'DKK')
print('- Solar to grid (x1) =', x1_component, 'DKK')
print('- Solar to battery (x2) =', x2_component, 'DKK')
print('- Battery to grid (x3) =', x3_component, 'DKK')
print('- Grid to battery (x4) =', x4_component, 'DKK')
print('- Check: ', x1_component + x2_component + x3_component + x4_component)
print()
print('Solar to grid:', x1.value.round(3), 'maximum:', max(x1.value.round(3)))
print('Solar to battery:', x2.value.round(3), 'maximum:', max(x2.value.round(3)))
print('Battery to grid:', x3.value.round(3), 'maximum:', max(x3.value.round(3)))
print('Grid to battery:', x4.value.round(3), 'maximum:', max(x4.value.round(3)))

print()
print()

Solar only = 46.44 DKK
Solar + arbitrage = 92.84 DKK
- Solar to grid (x1) = 33.09 DKK
- Solar to battery (x2) = -4.36 DKK
- Battery to grid (x3) = 92.77 DKK
- Grid to battery (x4) = -28.65 DKK
- Check:  92.85

Solar to grid: [0.    0.    0.    0.    0.    1.57  1.57  1.57  1.57  1.57  1.57  1.57
 1.57  0.    0.    0.    0.    0.    0.845 1.57  1.57  1.57  0.    0.
 0.    0.    0.    0.    0.    1.57  1.57  1.57  1.57  1.57  1.57  1.57 ] maximum: 1.57
Solar to battery: [1.57  1.57  1.57  1.57  1.57  0.    0.    0.    0.    0.    0.    0.
 0.    1.57  1.57  1.57  1.57  1.429 0.725 0.    0.    0.    1.57  1.57
 1.57  1.57  1.57  1.57  1.57  0.    0.    0.    0.    0.    0.    0.   ] maximum: 1.57
Battery to grid: [ 0.     0.     0.     0.     0.     0.    11.23   6.72   3.36   0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     3.215
 11.23   5.615  0.     0.     0.     0.     0.     0.     0.     0.
 11.23   6.8    3.4    1.7    0.85   0.425] maximum: 11.23
Grid to batter

In [88]:
#print(result)
# The optimal value for x is stored in `x.value`.
#print(x.value.round(2))

# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
#print(np.round(constraints[0].dual_value, 2))


#plt.bar(c, x.value)
#plt.show()

In [36]:
cp.cumsum(c).value

array([ 0.,  1.,  3.,  6., 10., 15., 21.])

In [34]:
cp.max(cp.cumsum(c)).value

array(21.)